# Importing Basic libraries

In [44]:
import cv2
import numpy as np
import sys
import os
import shutil
from os import listdir
from os.path import isfile, join

# Reading Data items of both classes

In [45]:

mypath = ".//home//images//"
filename = [f for f in listdir(mypath) if isfile(join(mypath,f))]

print(str(len(filename))+' images loaded')

def make_dir(directory):
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory)    

# Data Preprocessing.....

In [46]:
bio_count = 0
non_bio_count = 0
size = 150
train_image = []
train_labels = []
test_image = []
test_labels = []
train_size = 270
test_size = 130

bio_dir_train = './/home/crowd/train/bio/'
non_bio_dir_train = './/home/crowd/train/non_bio/'
bio_dir_val = './/home/crowd/valid/bio/'
non_bio_dir_val = './/home/crowd/valid/non_bio/'

make_dir(bio_dir_train)
make_dir(non_bio_dir_train)
make_dir(bio_dir_val)
make_dir(non_bio_dir_val)


for i,file in enumerate(filename):
    
    if(file[0] == 'b'):
        if(bio_count >= train_size+test_size):
            continue
        else:
            img=cv2.imread(mypath+file)
            img=cv2.resize(img,(size,size),interpolation=cv2.INTER_AREA)
            train_image.append(img)
            train_labels.append('0')
            cv2.imwrite(bio_dir_train+'bio'+str(i)+'.jpg',img)        
                
    if(file[0] == 'n'):
            img=cv2.imread(mypath+file)
            img=cv2.resize(img,(size,size),interpolation=cv2.INTER_AREA)
            train_image.append(img)
            train_labels.append('1')
            cv2.imwrite(non_bio_dir_train+'non_bio'+str(i)+'.jpg',img)

print('data loaded sucessfully')    

data loaded sucessfully


# Storing data arrays

In [34]:
np.savez('Waste_train_data.npz',np.array(train_image))
np.savez('Waste_train_labels.npz',np.array(train_labels))
np.savez('Waste_test_data.npz',np.array(test_image))
np.savez('Waste_test_labels.npz',np.array(test_labels))

In [35]:
def load_data(dataset):
    npzfile = np.load(dataset+'_train_data.npz')
    train = npzfile['arr_0']
    
    npzfile = np.load(dataset+'_train_labels.npz')
    train_labels = npzfile['arr_0']
    
    npzfile = np.load(dataset+'_test_data.npz')
    test = npzfile['arr_0']
    
    npzfile = np.load(dataset+'_test_labels.npz')
    test_labels = npzfile['arr_0']
    
    return (train,train_labels),(test,test_labels)

In [36]:
for i in range(1,11):
    val=np.random.randint(0,len(train_labels))
    #print(train_image[val])
    cv2.imshow("image_"+str(i),train_image[val])
    
    if(train_labels[val]=='0'):
        print('BIODEGRADABLE')
    else:
        print('NON BIODEGRADABLE')
    cv2.waitKey(0) 

cv2.destroyAllWindows()

BIODEGRADABLE
BIODEGRADABLE
BIODEGRADABLE
BIODEGRADABLE
NON BIODEGRADABLE
NON BIODEGRADABLE
NON BIODEGRADABLE
BIODEGRADABLE
NON BIODEGRADABLE
NON BIODEGRADABLE


# Data Normalization

In [37]:
(x_train,y_train),(x_test,y_test) = load_data('Waste')

y_train=y_train.reshape(y_train.shape[0],1)
y_test=y_test.reshape(y_test.shape[0],1)

x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train=x_train/255
x_test=x_test/255


# Building Keras Model 

This model based on Convolution Neural Network algorithum of Deep Learning

Architecture consist of :

1.) A convolution layer   
2.) Followed by Relu Activation Layer   
3.) Followed by MaxPooling Layer

In [38]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# Number of Data points in one epoch
batch_size = 16

# Total iterative epochs 
epochs = 15

# Organising the input Data Shape
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]
input_shape = (img_rows, img_cols, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Model compilation with loss as "binary crossentropy" and optimizer as "rmsprop"
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Visualizing our model Architecture 
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)        18496     
__________

# Executing Training of Model

In [39]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)



# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 519 samples, validate on 3 samples
Epoch 1/15
519/519 [==============================] - 21s 40ms/step - loss: 0.7264 - acc: 0.6050 - val_loss: 0.2753 - val_acc: 1.0000
Epoch 2/15
519/519 [==============================] - 20s 38ms/step - loss: 0.5312 - acc: 0.7418 - val_loss: 0.1233 - val_acc: 1.0000
Epoch 3/15
519/519 [==============================] - 20s 38ms/step - loss: 0.5108 - acc: 0.7514 - val_loss: 0.0605 - val_acc: 1.0000
Epoch 4/15
519/519 [==============================] - 19s 36ms/step - loss: 0.4783 - acc: 0.8035 - val_loss: 0.1889 - val_acc: 1.0000
Epoch 5/15
519/519 [==============================] - 20s 38ms/step - loss: 0.4550 - acc: 0.8150 - val_loss: 0.3679 - val_acc: 0.6667
Epoch 6/15
519/519 [==============================] - 21s 40ms/step - loss: 0.4802 - acc: 0.8478 - val_loss: 0.1536 - val_acc: 1.0000
Epoch 7/15
519/519 [==============================] - 21s 40ms/step - loss: 0.3289 - acc: 0.8748 - val_loss: 0.0537 - val_acc: 1.0000
Epoch 8/15
519/519

# Saving Model api for future use

In [40]:
model.save('Waste_95.h5')

# Model Testing on validation Data and cv2

In [43]:
from keras.models import load_model
import numpy as np

classifier = load_model('Waste_95.h5')

def draw_test(name, pred, input_im):
    BLACK = [0,0,0]
    if pred == "[[0]]":
        pred = "BIO"
    if pred == "[[1]]":
        pred = "NON BIO"
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0] ,
                                        cv2.BORDER_CONSTANT,value=BLACK)
    #expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (252, 70) , 
                cv2.FONT_HERSHEY_COMPLEX_SMALL,4, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


for i in range(0,10):
    val=np.random.randint(0,len(y_test))
    input_im=x_test[val]
    
    imageL=cv2.resize(input_im,None,fx=2,fy=2,interpolation=cv2.INTER_CUBIC)
    
    cv2.imshow('image',imageL)
    
    input_im = input_im.reshape(1,150,150,3) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0))
    
    draw_test("Prediction", res, imageL) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

# Model Tesing on Live frame on Real-Time Objects

In [47]:
from keras.models import load_model
import numpy as np
import cv2

classifier=load_model('Waste_95.h5')
def draw_text(name, pred, input_im):
    BLACK = [0,0,0]
    #expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0] ,
                                       # cv2.BORDER_CONSTANT,value=BLACK)
    #expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(input_im, pred, (3, 70) , 
                cv2.FONT_HERSHEY_COMPLEX_SMALL,2, (255,0,0), 3)
    cv2.imshow(name, input_im)

cam=cv2.VideoCapture(0)
while True:
    yes,frame=cam.read()

    frame1=frame
    frame=cv2.resize(frame,(150,150),interpolation=cv2.INTER_AREA)
    frame=frame.astype('float32')
    
    frame=frame/255
    
    frame=frame.reshape(1,150,150,3)
    res=str(classifier.predict_classes(frame,1,verbose=0))
    if(res=='[[0]]'):
        draw_text('final','BIODEGRADABLE',frame1)
    
    if(res=='[[1]]'):
        draw_text('final','NON_BIODEGRADABLE',frame1)
    if cv2.waitKey(1)==27:
        break

cam.release()
cv2.destroyAllWindows()
        